# **사전 준비**

In [ ]:
%%capture --no-stderr
%pip install -U langgraph
%pip install -U langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 실습에 사용할 그래프의 상태 정의
# TypedDict = 타입 힌트

# Annotated는 런타임에 실제 동작하지 않고, 타입 검사기와 프레임워크에게 ‘설명/주석’ 역할을 하는 문법
# → “messages는 list 타입이야, 그리고 add_messages라는 규칙이 붙어 있어”
# Python 입장에서는 단순 주석
# LangGraph 입장에서는 병합 방식 지시

class State(TypedDict):
  messages: Annotated[list, add_messages]

# 실습에서 사용할 그래프 인스턴스 생성
graph_builder = StateGraph(State)

# **챗봇 노드**

In [ ]:
from langchain_openai import ChatOpenAI

# 오픈 AI 클라이언트 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 오픈AI를 호출하여 응답을 받아온 뒤 상태값에 저장하여 반환하는 챗봇 함수 정의
def chatbot(state: State):
  return {"messages": [llm.invoke(state["messages"])]}

# 챗봇 노드 정의
graph_builder.add_node("chatbot", chatbot)

In [ ]:
from langgraph.graph import StateGraph, START, END

# 진입 지점
graph_builder.add_edge(START, "chatbot")

# 종료 지점
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()